# Store Sales: Data Visualization 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [ ]:
%run -n main.py

In [ ]:
df = pd.read_csv("../datasets/train.csv")
df.head()